Отображаем найденные совпадения используя встроенный метод cv2.ORB_create и cv2.BFMatcher

In [ ]:
import cv2

img1 = cv2.imread('2024_12_15_15_31_8_num_3/1734248113095.jpg', 0)
img2 = cv2.imread('2024_12_15_15_31_8_num_3/1734248113241.jpg', 0)

orb = cv2.ORB_create()
kp1, des1 = orb.detectAndCompute(img1, None)
print(len(kp1)) # Выводим количество ключевых точек
kp2, des2 = orb.detectAndCompute(img2, None)
print(len(kp2))

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)

print(len(matches))  # Выводим количество совпадений (matches)

# Рисуем соответствия
img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches, None)
cv2.imshow("Matches", img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import os

# Укажи путь к папке с изображениями
img_folder = '2024_12_15_15_31_8_num_3'

# Получаем список файлов с расширением .jpg и сортируем
img_files = sorted([f for f in os.listdir(img_folder) if f.lower().endswith('.jpg')])

# Инициализация ORB и BFMatcher
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Проходим по парам соседних изображений
for i in range(len(img_files) - 1):
    # Чтение двух последовательных изображений в режиме градаций серого
    img1_path = os.path.join(img_folder, img_files[i])
    img2_path = os.path.join(img_folder, img_files[i + 1])
    
    img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)
    
    # Если одно из изображений не загрузилось, пропускаем пару
    if img1 is None or img2 is None:
        continue
    
    # Вычисляем ключевые точки и дескрипторы для обеих картинок
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    
    # Если дескрипторы не найдены, пропускаем пару
    if des1 is None or des2 is None:
        continue
    
    # Поиск соответствий между дескрипторами
    matches = bf.match(des1, des2)
    
    # Сортируем найденные соответствия по расстоянию (чем меньше расстояние, тем лучше)
    matches = sorted(matches, key=lambda x: x.distance)
    
    # Можно ограничить число отрисовываемых соответствий, например, первыми 20
    num_matches_to_draw = 20
    img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches[:num_matches_to_draw], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    # Отображаем результат
    cv2.imshow("Matches", img_matches)
    
    # Ожидаем нажатия клавиши. Если нажата клавиша Esc (код 27), выходим.
    key = cv2.waitKey(0) & 0xFF
    if key == 27:
        break

cv2.destroyAllWindows()

Отображаем найденные совпадения используя XFeat

In [ ]:
import os
os.environ["QT_QPA_PLATFORM"] = "xcb"
import cv2

import numpy as np
import json

from modules.xfeat_ort import XFeat
import logging

# Настройка логгера
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='vio_ort.log',
    filemode='w'
)
logger = logging.getLogger()

# Загружаем параметры камеры
with open('fisheye_2024-09-18.json') as f:
    camparam = json.load(f)

# Формируем маску (по контуру объекта с label == 'mask')
for shape in camparam['shapes']:
    if shape['label'] == 'mask':
        MASK = np.zeros((camparam['imageHeight'], camparam['imageWidth'], 3), dtype=np.uint8)
        cnt = np.asarray(shape['points']).reshape(-1, 1, 2).astype(np.int32)
        cv2.drawContours(MASK, [cnt], -1, (255, 255, 255), -1)

# Глобальные переменные для преобразования
CENTER = [camparam['ppx'], camparam['ppy']]
CENTER[0] += -6   # TODO: поправка
CENTER[1] += 26   # TODO: поправка
FOCAL = camparam['focal']
RAD = camparam['radius']
CROP_CENTER = np.asarray([RAD / 2, RAD / 2])

# Функция для вычисления карты преобразования "рыбий глаз → прямолинейное"
def fisheye2rectilinear(focal, pp, rw, rh, fproj='equidistant'):
    rx, ry = np.meshgrid(np.arange(rw) - rw // 2, np.arange(rh) - rh // 2)
    r = np.sqrt(rx**2 + ry**2) / focal
    angle_n = np.arctan(r)
    if fproj == 'equidistant':
        angle_n = angle_n
    elif fproj == 'orthographic':
        angle_n = np.sin(angle_n)
    elif fproj == 'stereographic':
        angle_n = 2 * np.tan(angle_n / 2)
    elif fproj == 'equisolid':
        angle_n = 2 * np.sin(angle_n / 2)
    angle_t = np.arctan2(ry, rx)
    pt_x = focal * angle_n * np.cos(angle_t) + pp[0]
    pt_y = focal * angle_n * np.sin(angle_t) + pp[1]
    map_x = pt_x.astype(np.float32)
    map_y = pt_y.astype(np.float32)
    return map_x, map_y

# Простая функция для применения маски
def preprocess_frame(frame, mask):
    return np.where(mask, frame, 0)

# Инициализируем XFeat
matcher = XFeat(top_k=512, detection_threshold=0.05)

# Пути к двум изображениям
img1_path = '2024_12_15_15_31_8_num_3/1734248113095.jpg'
img2_path = '2024_12_15_15_31_8_num_3/1734248113241.jpg'

# Загружаем изображения (в цветном формате)
img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)

if img1 is None or img2 is None:
    logger.error("Не удалось загрузить одно из изображений.")
    raise SystemExit("Ошибка загрузки изображений")

# Применяем маску
img1_masked = preprocess_frame(img1, MASK)
img2_masked = preprocess_frame(img2, MASK)

# Выполняем преобразование из рыбьего глаза в прямолинейное.
# Здесь в качестве размеров берем RAD x RAD – можно менять в зависимости от задачи.
map_x, map_y = fisheye2rectilinear(FOCAL, CENTER, RAD, RAD)
img1_rect = cv2.remap(img1_masked, map_x, map_y,
                        interpolation=cv2.INTER_LINEAR,
                        borderMode=cv2.BORDER_CONSTANT)
img2_rect = cv2.remap(img2_masked, map_x, map_y,
                        interpolation=cv2.INTER_LINEAR,
                        borderMode=cv2.BORDER_CONSTANT)

# Приводим изображения к виду, который ожидает XFeat (в add_trace_pt в проекте используется parse_input)
img1_input = matcher.parse_input(img1_rect)
img2_input = matcher.parse_input(img2_rect)

# Вычисляем ключевые точки и дескрипторы
out1 = matcher.detectAndCompute(img1_input)[0]
out2 = matcher.detectAndCompute(img2_input)[0]

logger.debug(f"Изображение 1: {len(out1['keypoints'])} ключевых точек")
logger.debug(f"Изображение 2: {len(out2['keypoints'])} ключевых точек")

# Сопоставляем дескрипторы через XFeat.match
idxs1, idxs2 = matcher.match(out1['descriptors'], out2['descriptors'], min_cossim=-1)
logger.debug(f"Найдено сопоставлений: {len(idxs1)}")

# Для отрисовки через cv2.drawMatches нам нужно сформировать объекты cv2.KeyPoint.
# Функция для преобразования координат в список cv2.KeyPoint
def create_cv2_keypoints(pts):
    kps = []
    for pt in pts:
        # Предполагаем, что pt имеет вид (x, y)
        kps.append(cv2.KeyPoint(float(pt[0]), float(pt[1]), 1))
    return kps

# Получаем координаты ключевых точек по найденным индексам.
# Если тип ключевых точек – не numpy-массив, пробуем привести через np.array.
kp1_coords = list(out1['keypoints'])
kp2_coords = list(out2['keypoints'])
print(len(kp1_coords))
print(len(kp2_coords))
kp1_coords = [kp1_coords[i] for i in idxs1]
kp2_coords = [kp2_coords[i] for i in idxs2]

kps1 = create_cv2_keypoints(kp1_coords)
kps2 = create_cv2_keypoints(kp2_coords)

# Создаём объекты DMatch – здесь по количеству совпадений.
matches = []
for i in range(len(kps1)):
    # В DMatch поля _queryIdx и _trainIdx соответствуют индексам в списках ключевых точек
    m = cv2.DMatch(_queryIdx=i, _trainIdx=i, _distance=0)
    matches.append(m)

print(len(matches))

# Отрисовываем сопоставления на прямолинейных изображениях
img_matches = cv2.drawMatches(img1_rect, kps1,
                                img2_rect, kps2,
                                matches, None,
                                flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv2.imshow("Matches", img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()
